In [7]:
!pip install torch_geometric
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, InstanceNorm, GATConv
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import wandb
from tqdm import tqdm

In [12]:
train = np.load('/kaggle/input/gcn-data/embed_outputs_train.npy')
test = np.load('/kaggle/input/gcn-data/embed_outputs_test.npy')
adj_matrix = np.load('/kaggle/input/gcn-data/adj_matrix.npy')
labels = np.load('/kaggle/input/gcn-data/labels.npy')

temp = [x for x in adj_matrix]
mn = np.mean(temp)
adj_matrix[adj_matrix < mn] = 0

train_mask = [True]*708 + [False]*177
test_mask = [False]*708 + [True]*177
train_mask = np.array(train_mask)
test_mask = np.array(test_mask)

In [13]:
features = torch.from_numpy(np.concatenate((train, test), axis=0)).float()
labels = torch.from_numpy(labels).float()
edge_index = torch.from_numpy(np.argwhere(adj_matrix != 0).T).long()
data = Data(x=features, edge_index=edge_index, y=labels)
data.train_mask = torch.tensor(train_mask)
data.test_mask = torch.tensor(test_mask)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
def train(data):
    model.train()
    optimizer.zero_grad()
    data = data.to(device) 
    out = model(data)[data.train_mask]
    loss = criterion(out, data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(data):
    model.eval()
    with torch.no_grad():
        data = data.to(device) 
        pred = model(data).round()[data.test_mask]
        correct = pred.eq(data.y[data.test_mask]).sum().item()
        acc = correct / data.test_mask.sum().item()
    return acc, pred

In [18]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1d = torch.nn.Conv2d(1, 16, (3, 3))
        self.conv1 = GCNConv(6*16*30, 512)
        self.conv2 = GCNConv(512, 128)
        self.norm1 = InstanceNorm(512)  
        self.norm2 = InstanceNorm(128)  
        self.out = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(0.2) 
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = x.view(-1, 1, 8, 32)  
        x = F.relu(self.conv1d(x))
        x = x.view(data.num_nodes, -1)
        x = F.relu(self.conv1(x, edge_index))
        x = self.norm1(x)  
        x = self.dropout(x) 
        x = F.relu(self.conv2(x, edge_index))
        x = self.norm2(x) 
        x = self.dropout(x) 
        x = self.out(x)
        x = torch.sigmoid(x)
        return x.squeeze(1)

learning_rates = [0.001, 0.005, 0.0005]
arr = []
for i in range(708):
    if labels[i]==1:
        arr.append(1/len(labels[labels==1]))
    elif labels[i]==0:
        arr.append(1/len(labels[labels==0]))
weights = torch.tensor(arr).to(device)
criterions = {'BCE': torch.nn.BCELoss(), 'Weighted_BCE': torch.nn.BCELoss(weight=weights)}
bestest_f1 = {"learning_rates":None,"criterions":None}
    
for lr in tqdm(learning_rates, desc='Learning Rates'):
    for crit_name, criterion in criterions.items():
        wandb.init(project="gcn_experiments", config={"learning_rate": lr,"criterion": crit_name })

        model = GCN().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)   

        max_acc = 0
        max_f1 = 0
        best_epoch=0
        losses=[]
        accs=[]
        true_labels = data.y[data.test_mask].cpu().numpy()
        for epoch in range(1,5001):
            loss = train(data)
            acc,pred = test(data)
            losses.append(loss)
            accs.append(acc)
            predictions = pred.cpu().numpy()
            f1 = f1_score(true_labels, predictions)
            if f1 > max_f1:
                max_acc = acc
                best_epoch=epoch
                max_f1=f1
            wandb.log({
                "epoch": epoch,
                "loss": loss,
                "accuracy": acc})
            if epoch % 1000 == 0:
                print(f'Epoch: {epoch:03d}, Loss: {loss:.6f}, Acc: {acc:.4f}, Best F1: {max_f1:.5f}')

        wandb.log({"Learning Rate":lr, "Criterion":crit_name,
                "Maximum Test Accuracy": max_acc,
                "F1 Score": max_f1})

        wandb.finish()

Learning Rates:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1000, Loss: 0.451621, Acc: 0.8305, Best F1: 0.88496
Epoch: 2000, Loss: 0.410834, Acc: 0.8362, Best F1: 0.89778
Epoch: 3000, Loss: 0.376688, Acc: 0.8079, Best F1: 0.90749
Epoch: 4000, Loss: 0.364202, Acc: 0.8249, Best F1: 0.91228
Epoch: 5000, Loss: 0.352700, Acc: 0.8305, Best F1: 0.92444


F1 Score,▁
Learning Rate,▁
Maximum Test Accuracy,▁
accuracy,▃▁▄▄▅▄▅▄▅▅▅▅▆▇▆▅▅▇▆▆▆█▇▇▇▅██▇▇▇▇█▅█▇▇▅▅▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▄▄▃▄▃▃▃▃▃▂▃▂▂▂▃▃▂▂▁▁▃▂▂
Criterion,BCE
F1 Score,0.92444
Learning Rate,0.001
Maximum Test Accuracy,0.90395
accuracy,0.83051


Epoch: 1000, Loss: 0.403664, Acc: 0.8305, Best F1: 0.89593
Epoch: 2000, Loss: 0.371442, Acc: 0.8588, Best F1: 0.91403
Epoch: 3000, Loss: 0.332551, Acc: 0.8136, Best F1: 0.91403
Epoch: 4000, Loss: 0.315438, Acc: 0.8588, Best F1: 0.91403
Epoch: 5000, Loss: 0.281149, Acc: 0.8475, Best F1: 0.91630


F1 Score,▁
Learning Rate,▁
Maximum Test Accuracy,▁
accuracy,▁▁▂▄▆▆▄▃▅▅▆▆▆▆▆▆▆▆▇▇▇▆▆▆▆█▇▆▆▇▇█▇▆▆▇▇▇▇▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▆▅▅▆▅▅▅▄▄▄▄▃▄▄▃▃▃▃▄▃▃▂▂▃▂▂▂▂▃▂▁▁▁▂▁▂
Criterion,Weighted_BCE
F1 Score,0.9163
Learning Rate,0.001
Maximum Test Accuracy,0.89266
accuracy,0.84746


...

Epoch: 1000, Loss: 0.459325, Acc: 0.8249, Best F1: 0.88312
Epoch: 2000, Loss: 0.408113, Acc: 0.8362, Best F1: 0.90351
Epoch: 3000, Loss: 0.370819, Acc: 0.8418, Best F1: 0.90583
Epoch: 4000, Loss: 0.368475, Acc: 0.8644, Best F1: 0.91703
Epoch: 5000, Loss: 0.336661, Acc: 0.8305, Best F1: 0.91703


F1 Score,▁
Learning Rate,▁
Maximum Test Accuracy,▁
accuracy,▁▃▃▄▅▄▄▅▆▇▅▇▇█▅▇▆▇▆▅▆▇▇▇▆▆▆█▇█▇▇▇▇▇▇▆██▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▅▅▄▅▄▄▃▄▃▄▄▃▃▃▂▃▂▃▃▂▂▁▂▂▂▂▂▁▂▁▁▂▁▁▂
Criterion,BCE
F1 Score,0.91703
Learning Rate,0.0005
Maximum Test Accuracy,0.89266
accuracy,0.83051


Epoch: 1000, Loss: 0.455197, Acc: 0.8249, Best F1: 0.88789
Epoch: 2000, Loss: 0.413616, Acc: 0.8136, Best F1: 0.91630
Epoch: 3000, Loss: 0.386114, Acc: 0.8305, Best F1: 0.91630
Epoch: 4000, Loss: 0.347552, Acc: 0.8588, Best F1: 0.91703
Epoch: 5000, Loss: 0.340504, Acc: 0.8531, Best F1: 0.91964


F1 Score,▁
Learning Rate,▁
Maximum Test Accuracy,▁
accuracy,▂▁▂▁▄▄▅▆▄▄▇▆▅▄▄▆▆▇█▇▇▇▆▅▅▇▆▇▄▇▆█▇▇▆▇█▆▆▆
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▅▅▅▅▅▅▄▄▄▄▅▄▃▄▄▃▃▃▃▃▂▃▂▃▄▂▂▂▂▂▂▁▂▁▂▃
Criterion,Weighted_BCE
F1 Score,0.91964
Learning Rate,0.0005
Maximum Test Accuracy,0.89831
accuracy,0.85311


Learning Rates: 100%|██████████| 3/3 [41:05<00:00, 821.86s/it]
